In [25]:
import pandas as pd
import networkx as nx
from pyvis import network as net
from pyvis.network import Network

from serpapi import GoogleScholarSearch
import json
import requests

## ES 1
A partire da nodes.csv, utilizzare la libreria Python SerpAPI per scaricare, per ciascuno dei 7 autori elencati:
- author_id: ID identificativo del profilo Google Scholar
- cited_by: numero totale di citazioni ricevute
- interests: elenco degli interessi di ciascun autore
Sfruttando la libreria Python Pandas, usare la struttura dati DataFrame per aggiornare il file originale con apposite colonne e memorizzarlo nella cartella /data.

In [ ]:
#Serpapi keys

GoogleScholarSearch.SERP_API_KEY_FAVA = "1ae8a37d9b3dabb5732fcd7a9cb57410194766951242a4f4ac5bbd9fb7012048"
GoogleScholarSearch.SERP_API_KEY_LIGUGNANA = ""
GoogleScholarSearch.SERP_API_KEY_PARAVANO = ""
GoogleScholarSearch.SERP_API_KEY_TAMMARO = "6eaf5a9a0d2eef8bf026a947a3dffbb0d31cb1d6e122b81c0e6062f6b23a4c40"


In [71]:
df_nodes = pd.read_csv(r"Data/nodes.csv")
    
def list_articles(name, affiliation):
    base_url = "https://serpapi.com/search"
    
    params = {
        "api_key": "1ae8a37d9b3dabb5732fcd7a9cb57410194766951242a4f4ac5bbd9fb7012048",
        "engine": "google_scholar_profiles",
        "hl": "en",
        "mauthors": "David La Barbera"
    }      
    
    response = requests.get(base_url, params = params)
    result = response.json()
    
    authorID = result["profiles"][0]["author_id"] #save author_id for the next call

    if result["profiles"][0]["affiliations"] == affiliation: 
        #richimao  afunzione

    #verifies the affiliation
    
    
    
    #selezionare author_id
    

'Università Degli Studi Di Udine'

In [57]:
file = open('all_profiles_for_larry_page.json')
res = json.load(file)

res.keys()


dict_keys(['search_metadata', 'search_parameters', 'profiles'])

In [38]:
df_nodes = pd.read_csv(r"Data/nodes.csv")